In [1]:
import json
import ROOT
from analysis_framework import Dataset
from WWAnalysis import WWAnalysis
import os
from alt_setup_creator_new import AltSetupHandler

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x803e720


In [2]:
ROOT.EnableImplicitMT(12)

In [3]:
# CLD
# x_angle = 0.030 # rad
# ILD
x_angle = 0.014 # rad

In [4]:
prod = False
no_rvec = False
output_collections = ""
dataset_path = "stage2-test.json"
template_par_plot_path = "plots/template_parametrisation/test"
histo_path = "data/histograms/test"
output_path = "data/snapshots/stage3/test"
output_meta = "stage3-test.json"
if prod:
    dataset_path = "stage2-full.json"
    template_par_plot_path = "plots/template_parametrisation/full"
    histo_path = "data/histograms/full"
    output_path = "data/snapshots/stage3/full"
    output_meta = "stage3-full.json"

In [5]:
dataset = Dataset.from_json(dataset_path)

In [6]:
analysis = WWAnalysis(dataset)
analysis.init_categories()

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0xb3a2e50


In [7]:
alt_setup_handler = AltSetupHandler("alt-configs-new.json", mirror=True)
alt_configs = alt_setup_handler.get_alt_setup()
# alt_variations = alt_setup_handler.get_variations()

In [8]:
# check if we missed any processes
print(analysis.is_complete_categorisation())
signal_category = ["4f_sw_sl_signal"]
analysis.set_signal_categories(signal_category)

True


In [9]:
print(json.dumps(analysis._categories, indent=2))

{
  "4f_sw_sl_signal": [
    "4f_sw_sl_eLpR_signal",
    "4f_sw_sl_eRpL_signal"
  ],
  "4f_sl_bkg": [
    "4f_zz_sl_eLpR",
    "4f_sze_sl_eRpL",
    "4f_sze_sl_eLpR",
    "4f_sznu_sl_eLpR",
    "4f_sw_sl_eLpR_bkg"
  ],
  "4f_not_sl": [
    "4f_sw_l_eLpR",
    "4f_zz_h_eLpR",
    "4f_zz_l_eLpR",
    "4f_sznu_l_eLpR"
  ],
  "2f": [
    "2f_z_h_eLpR"
  ],
  "3f": [
    "ae_3f_z_l_eBpL"
  ],
  "6f": [
    "6f_llxyyx_eLpR",
    "6f_vvyyyy_eRpL",
    "6f_llyyyy_eRpL"
  ],
  "higgs": [
    "e1e1h_eLpR",
    "n1n1h_eLpR",
    "qqh_eLpR",
    "e2e2h_eLpR",
    "qqh_eRpL",
    "n1n1h_eRpL",
    "e1e1h_eRpR",
    "n23n23h_eRpL",
    "e1e1h_eLpL"
  ]
}


In [10]:
oo_setups = ["g1z_pos_1em05", "ka_pos_1em05", "la_pos_1em05", ]
analysis.define_optimal_observables(oo_setups)

In [11]:
n_bins = 65
oo_configs = {
    "O_g1z_pos_1em05": ("", "O_g1z_pos_1em05", n_bins, -3.5, 3.5),
    "O_ka_pos_1em05":  ("", "O_ka_pos_1em05", n_bins, -3.5, 3.5),
    "O_la_pos_1em05":  ("", "O_la_pos_1em05", n_bins, -3.5, 3.5),
}
for name, config in oo_configs.items():
    analysis.add_filter(f"{name} >= {config[-2]} && {name} <= {config[-1]}", f"{name} in range")
    analysis.book_histogram_1D(name, name, config)

In [12]:
conf_3d = ("", "", n_bins, -3.5, 3.5, n_bins, -3.5, 3.5, n_bins, -3.5, 3.5)
conf_nd_sparse = ("", "", [ROOT.TAxis(n_bins,  -3.5, 3.5) for i in range(3)])
conf_nd = ("", "", 3, [n_bins]*3, [-3.5] * 3, [3.5] * 3)
analysis.book_histogram_3D("obs", conf_3d, ["O_g1z_pos_1em05", "O_ka_pos_1em05", "O_la_pos_1em05"])
# fundamentally broken? :(
# analysis.book_histogram_ND_sparse("obs_sparse", conf_nd_sparse, ["O_g1z_pos_1em05", "O_ka_pos_1em05", "O_la_pos_1em05"])
analysis.book_histogram_ND("obs_ND", conf_nd, ["O_g1z_pos_1em05", "O_ka_pos_1em05", "O_la_pos_1em05"])

In [13]:
weights = [f"weight_{name}" for name in ["nominal"] + list(alt_configs.keys())]
# analysis._define(("weight_nominal", "1.0"), signal_category)
analysis.book_templates(oo_configs, weights, signal_category)
analysis.book_oo_matrix(list(oo_configs.keys()))

In [14]:

# analysis.book_snapshots("events", output_path, output_meta, output_collections, no_rvec=no_rvec)

In [15]:
for oo in oo_configs:
    analysis.book_sum(oo, oo, signal_category)
    for oo2 in oo_configs:
        analysis.define_only_on(signal_category, f"{oo}_{oo2}", f"{oo} * {oo2}")
        analysis.book_sum(f"{oo}_{oo2}", f"{oo}_{oo2}", signal_category)

In [16]:
analysis.book_reports()

In [17]:
%%time
analysis.run()

CPU times: user 3.53 s, sys: 122 ms, total: 3.65 s
Wall time: 1.88 s


In [18]:
analysis.print_reports()

       4f_sw_sl_signal             4f_sl_bkg             4f_not_sl                    2f                    3f                    6f                 higgs
         9739614 (4e+04)           29300 (1e+03)           27051 (1e+03)           42094 (2e+04)            1399 (1e+03)               1 (3e-01)           13526 (8e+02) All
         9641631 (4e+04)           27375 (9e+02)           27033 (1e+03)           42094 (2e+04)            1399 (1e+03)               1 (3e-01)           13352 (8e+02) O_g1z_pos_1em05 in range
         9618764 (4e+04)           26276 (9e+02)           27027 (1e+03)           42094 (2e+04)            1399 (1e+03)               1 (3e-01)           13335 (8e+02) O_ka_pos_1em05 in range
         9439684 (4e+04)           25105 (9e+02)           26932 (1e+03)           42094 (2e+04)            1399 (1e+03)               1 (3e-01)           13144 (8e+02) O_la_pos_1em05 in range
            0.97            0.86            1.00            1.00            1.00            

In [19]:

for name in oo_configs:
    # analysis.draw_histogram(name, categories=signal_category)
    analysis.draw_histogram(name)

In [20]:
# analysis.plot_template_bins("O_g1z_pos_1em05", alt_setup_handler, only=oo_setups)
analysis.calculate_template_parametrisation_full("O_g1z_pos_1em05", alt_setup_handler)
# analysis.plot_template_bins("O_g1z_pos_1em05", plot_path=template_par_plot_path)
analysis.calculate_template_parametrisation_full("O_ka_pos_1em05", alt_setup_handler)
# analysis.plot_template_bins("O_ka_pos_1em05", plot_path=template_par_plot_path)
analysis.calculate_template_parametrisation_full("O_la_pos_1em05", alt_setup_handler)
# analysis.plot_template_bins("O_la_pos_1em05", plot_path=template_par_plot_path)

x[0] * [p_0] + x[1] * [p_1] + x[2] * [p_2] + x[0] * x[0] * [p_3] + x[0] * x[1] * [p_4] + x[0] * x[2] * [p_5] + x[1] * x[1] * [p_6] + x[1] * x[2] * [p_7] + x[2] * x[2] * [p_8]
[[-0.1681571640163474, -2.025463441723785, 1.5854040719047977, 68.45308947393818, -7.566825513705428, -9.261746602309028, 60.36029205255247, -1.0494503793144347, 63.48643003113966], [0.5252339064910754, -0.3870598386615886, 1.648780286830869, 59.31085949857652, -2.562292552472987, 2.3152621429823097, 57.99393182197885, 1.4070692862520366, 63.946311923673065], [1.4316219530044072, 1.1741519613381417, -0.5454275301287119, 60.08710980317117, 4.207628854742619, -4.808863334803603, 58.1678718509272, -3.0308817157851533, 58.823454669313115], [1.386195702240377, -0.7643403514362496, 1.1090498224051857, 63.59959906193615, -7.469126993280841, 2.3352287841806576, 60.348443846666974, 1.9292442497210192, 64.82117646730163], [1.9258172923462564, 0.46316853216529785, -0.23536534828405445, 100.30155850613532, 32.55141113004888, 

In [21]:
# TODO: store the 3d matrix...
analysis.write_fit_inputs(list(oo_configs.keys()), histo_path)
analysis.store_raw_histograms(["obs_ND"], histo_path, opening_option="update")

In [22]:
def make_observable(h):
    res = 0.
    for i in range(h.GetNbinsX()):
        bin_content = h.GetBinContent(i+1)
        bin_center = h.GetBinCenter(i+1)
        # print(f"i: {i} bin_center: {bin_center}, bin_content: {bin_content}")
        res += bin_center * bin_content
    return res

obs_binned = []
for oo in oo_configs:
    stack, *_ = analysis.draw_histogram(oo, categories=signal_category)
    h = stack.GetHists().First()
    obs_binned.append(make_observable(h))


In [23]:
import numpy as np
obs = []
cov = []
for oo in oo_configs:
    obs.append(analysis.get_sum(oo, categories=signal_category))
    for oo2 in oo_configs:
        name = f"{oo}_{oo2}"
        cov.append(analysis.get_sum(name, categories=signal_category))

Cov = np.asarray(cov).reshape((3,3))
print(Cov)
uncut = np.asarray([7.1362e+06, 1.6882e+06, -3.90017e+06, 1.6882e+06, 4.43426e+06, 98588.2, -3.90017e+06, 98588.2, 1.36143e+07]).reshape((3,3))
print("Cov / uncut")
print(Cov / uncut)
asimov = np.asarray([4520086.37641232, 1109138.80145051, -1998222.20804277, 1109138.80145051, 3657043.91937568, 207762.12386119, -1998222.20804277, 207762.12386119, 7502627.88897558]).reshape((3,3))
print("Cov / asimov")
print(Cov / asimov)

obs_np = np.asarray(obs)
obs_binned_np = np.asarray(obs_binned)
print("obs unbinned, binned, ratio")
print(obs_np)
print(obs_binned_np)
print(obs_np / obs_binned_np)

[[ 3947119.68389253   950076.90723365 -1965306.24008642]
 [  950076.90723365  2666305.90773936   386096.15631659]
 [-1965306.24008642   386096.15631659  7087821.06613641]]
Cov / uncut
[[0.55311226 0.56277509 0.50390271]
 [0.56277509 0.6012967  3.9162512 ]
 [0.50390271 3.9162512  0.5206159 ]]
Cov / asimov
[[0.87323988 0.85658973 0.98352737]
 [0.85658973 0.72908775 1.8583568 ]
 [0.98352737 1.8583568  0.9447118 ]]
obs unbinned, binned, ratio
[-303362.77893593 -557014.31722219 -366016.17588526]
[-199651.75023094 -468961.06533863 -366591.86193907]
[1.51945965 1.18776239 0.99842963]


In [24]:
c1 = ROOT.TCanvas()
analysis._template_pars["O_g1z_pos_1em05"]["4f_sw_sl_eLpR_signal"]["g1z"].Draw()
c1.Draw()

c2 = ROOT.TCanvas()
analysis._template_pars["O_g1z_pos_1em05"]["4f_sw_sl_eLpR_signal"]["g1z_g1z"].Draw()
c2.Draw()

c3 = ROOT.TCanvas()
analysis._template_pars["O_g1z_pos_1em05"]["4f_sw_sl_eLpR_signal"]["g1z_ka"].Draw()
c3.Draw()